## Lecture CSV

In [132]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report, confusion_matrix

In [133]:
# data_train = pd.read_csv("../datas/train_clean.csv")
data_train = pd.read_csv("/content/train_clean.csv")

data_train.head()

,keyword,location,text,labels
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


---

## distilbert-base-uncased

In [135]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

## jy46604790/Fake-News-Bert-Detect

In [91]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

## hamzab/roberta-fake-news-classification

In [105]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [136]:
def tokenize_function(example):
    return tokenizer(example["all_concat"], truncation=True, padding=True)

Rempace les NaN pour pouvoir l'utiliser

In [137]:
data_train["keyword"] = data_train["keyword"].fillna("")

In [138]:
data_train["location"] = data_train["location"].fillna("")

Concatène toutes les colonnes

In [139]:
data_train["all_concat"] = data_train["text"] + " " + data_train["keyword"] + " " + data_train["location"]

In [140]:
data_train.head()

,keyword,location,text,labels,all_concat
0,,,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...
1,,,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,,,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...
3,,,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,,,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...


Split train / val

In [141]:
from datasets import Dataset

# Conversion pandas -> Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data_train)

# Split 80% train, 20% test (ou val)
split_dataset = hf_dataset.train_test_split(test_size=0.2)

# Accès aux sous-datasets
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Appliquer la fonction de tokenization avec batching
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [142]:
from transformers import DataCollatorWithPadding

# Auto Padding : toutes les séquences sont de la même longueur
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [143]:
# On vérifie la taille des séquences
samples = tokenized_train_dataset[:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[84, 84, 84, 84, 84, 84, 84, 84]

---

## Fine Tuning

In [144]:
from transformers import TrainingArguments

# paramètres d'entrainement par défaut
training_args = TrainingArguments("test-trainer", report_to="none", num_train_epochs=10)

In [145]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
)

---

## Train

In [146]:
trainer.train()

Step,Training Loss
500,0.511000
1000,0.376000
1500,0.328900
2000,0.225000
2500,0.186000
3000,0.138000


Step,Training Loss
500,0.511000
1000,0.376000
1500,0.328900
2000,0.225000
2500,0.186000
3000,0.138000
3500,0.089700
4000,0.090600
4500,0.078800
5000,0.055300


TrainOutput(global_step=7620, training_loss=0.14882220076763725, metrics={'train_runtime': 1098.2224, 'train_samples_per_second': 55.453, 'train_steps_per_second': 6.938, 'total_flos': 1427046965534616.0, 'train_loss': 0.14882220076763725, 'epoch': 10.0})

---

## Predictions

In [147]:
predictions = trainer.predict(tokenized_val_dataset)

In [148]:
preds = np.argmax(predictions.predictions, axis=-1)

---

## Classification report

In [151]:
print(classification_report(tokenized_val_dataset["labels"], preds))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       872
           1       0.77      0.77      0.77       651

    accuracy                           0.80      1523
   macro avg       0.80      0.80      0.80      1523
weighted avg       0.80      0.80      0.80      1523



---

## Confusion Matrix

In [150]:
# produce the confusion matrix for your predictions, what comments can you make ?
mat = confusion_matrix(tokenized_val_dataset["labels"], preds)

labels = data_train["labels"].unique()
df_mat = pd.DataFrame(mat, index=labels, columns=labels)

fig = px.imshow(df_mat, text_auto=True, color_continuous_scale=px.colors.sequential.Aggrnyl,
    labels=dict(x="Prédiction", y="Réalité", color="Nombre"))
fig.update_coloraxes(showscale=False)
fig.show()